## Library imports

In [1]:
# basic imports
import os
import gc
import math
import glob
import random
import itertools
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# DL library imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from  torch.cuda.amp import autocast, GradScaler

# metrics calculation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold

# basic plotting library
import matplotlib.pyplot as plt

# interactive plots
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

import warnings  
warnings.filterwarnings('ignore')

## Config parameters

In [2]:
class CFG:
    # pipeline parameters
    SEED        = 42
    TRAIN       = True
    LR_FIND     = False
    GENERATE_OOF= True
    N_FOLDS     = 5 
    N_EPOCHS    = 2
    TEST_BATCH_SIZE  = 32
    TRAIN_BATCH_SIZE = 16
    NUM_WORKERS      = 4
    DATA_FRAC        = 1.0
    FOLD_TO_TRAIN    = [0, 1, 2, 3, 4] # 

    # model parameters
    MODEL_ARCH  = 'MLP'
    MODEL_NAME  = 'mlp_vtest'
    WGT_PATH    = ''
    WGT_MODEL   = ''
    PRINT_N_EPOCH = 2
    
    # scheduler variables
    MAX_LR    = 1e-2
    MIN_LR    = 1e-5
    SCHEDULER = 'CosineAnnealingWarmRestarts'  # ['ReduceLROnPlateau', 'None', OneCycleLR', ','CosineAnnealingLR']
    T_0       = 10     # CosineAnnealingWarmRestarts
    T_MULT    = 2      # CosineAnnealingWarmRestarts
    T_MAX     = 5      # CosineAnnealingLR

    # optimizer variables
    OPTIMIZER     = 'Adam'
    WEIGHT_DECAY  = 1e-6
    GRD_ACC_STEPS = 1
    MAX_GRD_NORM  = 1000
    
    # features parameters
    USE_FREQ_FEATS = True
    BUILDING_SITES_RANGE = [0,1]

In [3]:
floor_map = {"B2": -2, "B1": -1, "F1": 0, "F2": 1, "F3": 2, "F4": 3, "F5": 4, "F6": 5, "F7": 6, "F8": 7, "F9": 8,
             "1F": 0, "2F": 1, "3F": 2, "4F": 3, "5F": 4, "6F": 5, "7F": 6, "8F": 7, "9F": 8}

minCount = 1
rssiFillerValue = -999.0
dtFillerValue   = 1000.0
freqFillerValue = 0
featuresInputDir = 'referencePublicNotebooks/wiFiFeatures'
modelOutputDir = 'modelSaveDir'
sampleCsvPath = 'sample_submission.csv'

In [4]:
#buildingsList = sorted(glob.glob(f"{outputDir}/*.csv"))
#buildingsList = buildingsList[CFG.BUILDING_SITES_RANGE[0]: CFG.BUILDING_SITES_RANGE[1]]
#print(buildingsList[0].split('/')[-1])
buildingsList = 'referencePublicNotebooks/wiFiFeatures/5a0546857ecc773753327266_npyTrain.csv'

In [5]:
buildingData = pd.read_csv(buildingsList)

## Helper functions

In [10]:
def getBuildingName(buildingCsvPath):
    fileName = buildingCsvPath.split('/')[-1]
    buildingName = fileName.split('_')[0]
    return buildingName

In [ ]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_trainable_params

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(CFG.SEED)

In [17]:
def getBuilding_npyInfo(buildingCsvPath):
    # read building data 
    data = pd.read_csv(buildingCsvPath)
    buildingName = getBuildingName(buildingCsvPath)
    #print(buildingName)
        
    # use fraction if needed
    if CFG.DATA_FRAC < 1:
        data = data.sample(frac=CFG.DATA_FRAC).reset_index(drop=True)

    # first column is timestamp
    timestamps = data.iloc[:,0].values   # np.expand_dims( , ,axis=1)
    
    # last column is pathFile name
    groups = data.iloc[:,-1].values
    
    # target values are last but 3 columns
    y = data.iloc[:,-4:-1].values
    
    data['filePath'] = buildingName + '_' + \
                       data.iloc[:,-1].astype(str) + '_' + \
                       data.iloc[:,0].astype(str) + '.npy'
    X = data['filePath'].values
    del data
    gc.collect()
    return timestamps,X,y,groups

In [ ]:
def getBuildingData(buildingCsvPath):
    # read building data 
    data = pd.read_csv(buildingCsvPath)
    
    # use fraction if needed
    if CFG.DATA_FRAC < 1:
        data = data.sample(frac=CFG.DATA_FRAC).reset_index(drop=True)

    # first column is timestamp
    timestamps = data.iloc[:,0].values   # np.expand_dims( , ,axis=1)
    
    # last column is pathFile name
    groups = data.iloc[:,-1].values
    
    # target values are last but 3 columns
    y = data.iloc[:,-4:-1].values
    
    # use all features
    if CFG.USE_FREQ_FEATS == True:
        X = data.iloc[:,1:-4].values    

    else:
        numWiFiIds = int((data.shape[1] - 5) / 3)
        # separate into features and target variables
        X = data.iloc[:,1:(2*numWiFiIds)+1].values    
        
    """
    # Incase freq signal is not needed, use rssi and dt features alone    
    # There are 5 columns for timestamp, y, pathNames values in csv, reamining are features
    # total features = 3 * [rssi, dt, freq]
    # hence unique wifi ids = totalFeatures / 3
    """
            
    del data
    gc.collect()
    return timestamps,X,y,groups

In [ ]:
def competitionMetric(preds, targets):
    """ The metric used in this competition """
    # position error
    meanPosPredictionError = torch.mean(torch.sqrt(
                             torch.square(torch.subtract(preds[:,0], targets[:,0])) + 
                             torch.square(torch.subtract(preds[:,1], targets[:,1]))))
    # error in floor prediction
    meanFloorPredictionError = torch.mean(15 * torch.abs(preds[:,2] - targets[:,2]))
    return meanPosPredictionError, meanFloorPredictionError

In [ ]:
def getOptimizer(model : nn.Module):    
    if CFG.OPTIMIZER == 'Adam':
        optimizer = optim.Adam(model.parameters(), weight_decay=CFG.WEIGHT_DECAY, lr=CFG.MAX_LR)
    else:
        optimizer = optim.SGD(model.parameters(), weight_decay=CFG.WEIGHT_DECAY, lr=CFG.MAX_LR, momentum=0.9)
    return optimizer

In [ ]:
def getScheduler(optimizer, dataloader_train):
    if CFG.SCHEDULER == 'OneCycleLR':
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr= CFG.MAX_LR, epochs = CFG.N_EPOCHS, 
                          steps_per_epoch = len(dataloader_train), pct_start=0.25, div_factor=10, anneal_strategy='cos')
    elif CFG.SCHEDULER == 'CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=CFG.T_MULT, eta_min=CFG.MIN_LR, last_epoch=-1)
    elif CFG.SCHEDULER == 'CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_MAX * len(dataloader_train), eta_min=CFG.MIN_LR, last_epoch=-1)
    else:
        scheduler = None
    return scheduler

In [ ]:
def getFoldDataLoaders(timestamps,X,y,groups,trainIndex,validIndex):
    
    # splitting into train and validataion sets
    trainTimeStamps, X_train, y_train, trainGroups = timestamps[trainIndex], X[trainIndex], y[trainIndex], groups[trainIndex]
    validTimeStamps, X_valid, y_valid, validGroups = timestamps[validIndex], X[validIndex], y[validIndex], groups[validIndex] 
    
    # normalize input            
    #print(f"Before stdscaler : train_mean{X_train.mean(), X_train.std(), X_valid.mean(), X_valid.std()}")
    X_train = stdScaler.fit_transform(X_train)
    X_valid = stdScaler.transform(X_valid)
    #print(f"After stdscaler : train_mean{X_train.mean(), X_train.std(), X_valid.mean(), X_valid.std()}")
    #print(f"x,y shapes = {X_train.shape, y_train.shape, X_valid.shape, y_valid.shape}")
                        
    # create torch Datasets and Dataloader for each fold's train and validation data
    dataset_train = wiFiFeaturesDataset(trainTimeStamps, X_train, y_train, trainGroups)
    dataset_valid = wiFiFeaturesDataset(validTimeStamps, X_valid, y_valid, validGroups)            
    dataloader_train = DataLoader(dataset_train, batch_size= CFG.TRAIN_BATCH_SIZE, shuffle=True,
                              num_workers=CFG.NUM_WORKERS, pin_memory=False, drop_last=False)
    dataloader_valid = DataLoader(dataset_valid, batch_size= CFG.TEST_BATCH_SIZE, shuffle=True,
                              num_workers=CFG.NUM_WORKERS, pin_memory=False, drop_last=False)
    return dataloader_train, dataloader_valid

In [ ]:
def plotTrainingResults(resultsDf):
    # subplot to plot
    fig = make_subplots(rows=1, cols=1)
    colors = [ ('#d32f2f', '#ef5350'), ('#303f9f', '#5c6bc0'), ('#00796b', '#26a69a'),
                ('#fbc02d', '#ffeb3b'), ('#5d4037', '#8d6e63')]

    # find number of folds input df
    numberOfFolds = resultsDf['fold'].nunique()
    
    # iterate through folds and plot
    for i in range(numberOfFolds):
        data = resultsDf[resultsDf['fold'] == i]
        fig.add_trace(go.Scatter(x=data['epoch'].values, y=data['trainPosLoss'].values,
                                mode='lines', visible='legendonly' if i > 0 else True,
                                line=dict(color=colors[i][0], width=2),
                                name='trainPossLoss -Fold{}'.format(i)),row=1, col=1)

        fig.add_trace(go.Scatter(x=data['epoch'], y=data['valPosLoss'].values,
                                 mode='lines+markers', visible='legendonly' if i > 0 else True,
                                 line=dict(color=colors[i][1], width=2),
                                 name='valPosLoss -Fold{}'.format(i)),row=1, col=1)
    fig.show()

## Dataset class

In [ ]:
class wiFiFeaturesDataset(Dataset):
    def __init__(self, timeStamps, X_data, y_data, groups):
        self.timeStamps = timeStamps 
        self.X_data = X_data
        self.y_data = y_data
        self.groups = groups
        
    def __getitem__(self, index):
        x  = torch.from_numpy(self.X_data[index].astype(np.float32))
        y  = torch.from_numpy(self.y_data[index].astype(np.float32))
        ts = self.timeStamps[index].astype(np.int64)
        group = self.groups[index]
        return ts,x,y,group
    
    def __len__ (self):
        return len(self.X_data)

In [ ]:
class npyWiFiFeaturesDataset(Dataset):
    def __init__(self, timeStamps, X_data, y_data, groups):
        self.timeStamps = timeStamps 
        self.X_data = X_data
        self.y_data = y_data
        self.groups = groups
        
    def __getitem__(self, index):
        ts = self.timeStamps[index].astype(np.int64)
        x = torch.from_numpy(np.load(self.X_data[index]))
        y  = torch.from_numpy(self.y_data[index].astype(np.float32))
        group = self.groups[index]
        return ts,x,y,group
    
    def __len__ (self):
        return len(self.X_data)

## MLP Model class

In [ ]:
class wiFiFeaturesMLPModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        self.lin1 = nn.Linear(in_features=n_input, out_features=512)
        self.lin2 = nn.Linear(in_features=512,     out_features=32)
        self.lin3 = nn.Linear(in_features=32,      out_features=n_output)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(32)
        self.drops = nn.Dropout(0.3)        

    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn1(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = self.lin3(x)
        return x

## Compute Device as CPU or GPU

In [ ]:
## Device as cpu or tpu
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device('cpu')
print(device)

## Preprocessing classes

In [ ]:
# for cv
folds = GroupKFold(n_splits=CFG.N_FOLDS)

# for normalizing input data
stdScaler = StandardScaler()

# scaler to handle AMP
scaler = GradScaler()   

criterion = competitionMetric

## Lr range finder

In [ ]:
def plot_lr_finder_results(lr_finder): 
    # Create subplot grid
    fig = make_subplots(rows=1, cols=2)
    # layout ={'title': 'Lr_finder_result'}
    
    # Create a line (trace) for the lr vs loss, gradient of loss
    trace0 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['smooth_loss'],name='log_lr vs smooth_loss')
    trace1 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['grad_loss'],name='log_lr vs loss gradient')

    # Add subplot trace & assign to each grid
    fig.add_trace(trace0, row=1, col=1);
    fig.add_trace(trace1, row=1, col=2);
    iplot(fig, show_link=False)
    #fig.write_html(CFG.MODEL_NAME + '_lr_find.html');

In [ ]:
def find_lr(model, optimizer, data_loader, init_value = 1e-8, final_value=100.0, beta = 0.98, num_batches = 200):
    assert(num_batches > 0)
    mult = (final_value / init_value) ** (1/num_batches)
    lr = init_value
    optimizer.param_groups[0]['lr'] = lr
    batch_num = 0
    avg_loss = 0.0
    best_loss = 0.0
    smooth_losses = []
    raw_losses = []
    log_lrs = []
    dataloader_it = iter(data_loader)
    progress_bar = tqdm(range(num_batches))                
        
    for idx in progress_bar:
        batch_num += 1
        try:
            _, inputs, targets, _ = next(dataloader_it)
            #print(images.shape)
        except:
            dataloader_it = iter(data_loader)
            _, inputs, targets, _ = next(dataloader_it)

        # Move input and label tensors to the default device
        inputs = inputs.to(device)
        targets = targets.to(device)

        # handle exception in criterion
        try:
            # Forward pass
            y_preds = model(inputs)
            posLoss, floorLoss = criterion(y_preds, targets)
            loss = posLoss + floorLoss
        except:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results 
                    
        #Compute the smoothed loss
        avg_loss = beta * avg_loss + (1-beta) *loss.item()
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 50 * best_loss:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results
        
        #Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
        
        #Store the values
        raw_losses.append(loss.item())
        smooth_losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print info
        progress_bar.set_description(f"loss:{loss.item()},smoothLoss: {smoothed_loss},lr:{lr}")

        #Update the lr for the next step
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
    
    grad_loss = np.gradient(smooth_losses)
    lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                         'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
    return lr_finder_results

In [ ]:
if CFG.LR_FIND == True:
    # create dataset instance
    tempTs, tempX, tempY,_ = getBuildingData(buildingCsvPath=buildingsList[0])
    tempX = stdScaler.fit_transform(tempX)
    tempTrainDataset = wiFiFeaturesDataset(tempTs, tempX, tempY)
    tempTrainDataloader = DataLoader(tempTrainDataset, batch_size= CFG.TRAIN_BATCH_SIZE, shuffle=True,
                          num_workers=CFG.NUM_WORKERS, pin_memory=False, drop_last=False)
    
    # create model instance   
    model = wiFiFeaturesMLPModel(n_input=tempX.shape[1], n_output=3)
    model.to(device);
    
    # optimizer function, lr schedulers and loss function
    optimizer = getOptimizer(model)
    lrFinderResults = find_lr(model, optimizer, tempTrainDataloader)
    plot_lr_finder_results(lrFinderResults)
    del tempX, tempY, tempTrainDataset, tempTrainDataloader, model, optimizer

## Train & Validate helper functions

In [ ]:
def validateModel(model, validationDataloader):
    # placeholders to store output
    val_ts = []
    val_preds = []
    val_targets = []
    val_groups = []

    # set model to Validate mode
    model.eval()
    dataLoaderIterator = iter(validationDataloader)

    for idx in range(len(validationDataloader)):
        try:
            ts, inputs, targets, valGroups = next(dataLoaderIterator)
        except StopIteration:
            dataLoaderIterator = iter(validationDataloader)
            ts, inputs, targets, valGroups = next(dataLoaderIterator)

        inputs = inputs.to(device)
        targets = targets.to(device) 

        # forward prediction
        with torch.no_grad():    
            y_preds = model(inputs)

        # store predictions and targets to compute metrics later
        val_ts.append(ts)
        val_preds.append(y_preds)
        val_targets.append(targets)
        val_groups.append(valGroups)

    # concatenate to get as 1 2d array and find total loss  
    val_preds = torch.cat(val_preds, 0)
    val_targets = torch.cat(val_targets, 0)
    valPosLoss, valFloorLoss = criterion(val_preds, val_targets)
    valScore = valPosLoss #+ valFloorLoss

    # np array concatenation
    val_ts = np.concatenate(val_ts, axis=0)
    val_groups = np.concatenate(val_groups, axis=0)
    
    # store results
    validationResults = {'valPosLoss': valPosLoss.item() , 'valFloorLoss': valFloorLoss.item(),\
                         'val_ts': val_ts, 'val_groups': val_groups,
                         'val_preds'  :val_preds.cpu().data.numpy(), 
                         'val_targets':val_targets.cpu().data.numpy(),
                         }
    return validationResults

In [ ]:
def trainValidateOneFold(buildingName, i_fold, model, optimizer, scheduler, dataloader_train, dataloader_valid):
    trainFoldResults = []
    bestValScore = np.inf
    bestEpoch = 0

    for epoch in range(CFG.N_EPOCHS):
        #print('Epoch {}/{}'.format(epoch + 1, CFG.N_EPOCHS))
        model.train()
        trainPosLoss = 0.0
        trainFloorLoss = 0.0

        # training iterator
        tr_iterator = iter(dataloader_train)

        for idx in range(len(dataloader_train)):
            try:
                _, inputs, targets, _ = next(tr_iterator)
            except StopIteration:
                tr_iterator = iter(dataloader_train)
                _, inputs, targets, _ = next(tr_iterator)

            inputs = inputs.to(device)
            targets = targets.to(device)  

            # builtin package to handle automatic mixed precision
            with autocast():
                # Forward pass
                y_preds = model(inputs)   
                posLoss, floorLoss = criterion(y_preds, targets)
                loss = posLoss # + floorLoss

                # Backward pass
                scaler.scale(loss).backward()        
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad() 

                # log the necessary losses
                trainPosLoss   += posLoss.item()
                trainFloorLoss += floorLoss.item()

                if scheduler is not None: 
                    if CFG.SCHEDULER == 'CosineAnnealingWarmRestarts':
                        scheduler.step(epoch + idx / len(dataloader_train)) 
                    # onecyle lr scheduler / CosineAnnealingLR scheduler
                    else:
                        scheduler.step()
                    
        # Validate
        foldValidationResults = validateModel(model, dataloader_valid)
         
        # store results
        trainFoldResults.append({ 'fold': i_fold, 'epoch': epoch, 
                                  'trainPosLoss': trainPosLoss / len(dataloader_train), 
                                  'trainFloorLoss': trainFloorLoss / len(dataloader_train), 
                                  'valPosLoss'  : foldValidationResults['valPosLoss'] , 
                                  'valFloorLoss': foldValidationResults['valFloorLoss']})
        
        valScore = foldValidationResults['valPosLoss'] # + foldVal['valFloorLoss']
        # save best models        
        if(valScore < bestValScore):
            # reset variables
            bestValScore = valScore
            bestEpoch = epoch

            # save model weights
            torch.save({'model': model.state_dict(), 'val_ts' : foldValidationResults['val_ts'], 
                        'val_preds':foldValidationResults['val_preds'], 
                        'val_targets':foldValidationResults['val_targets'],
                        'val_groups' : foldValidationResults['val_groups']}, 
                        f"{modelOutputDir}/{buildingName}_{CFG.MODEL_NAME}_fold{i_fold}_epoch{epoch}.pth")

    print(f"For Fold {i_fold}, Best validation score of {bestValScore} was got at epoch {bestEpoch}") 
    #temp = pd.DataFrame(trainFoldResults)
    #print(temp.shape, list(temp.columns))
    return trainFoldResults

In [ ]:
def trainValidateOneBuilding(buildingDataPath):
    # placeholder to store results
    buildingTrainResults = []
    
    buildingName = getBuildingName(buildingDataPath)
    print(f"Processing data for building - {buildingName}")
    timestamps, X, y, groups = getBuildingData(buildingCsvPath=buildingDataPath)
    print(f"Building Data shapes : {timestamps.shape, X.shape, y.shape, groups.shape}")

    for i_fold, (trainIndex, validIndex) in enumerate(folds.split(X=X, y=y[:,0],groups=groups)):
        if i_fold in CFG.FOLD_TO_TRAIN:
            print("Fold {}/{}".format(i_fold + 1, CFG.N_FOLDS))
            dataloader_train, dataloader_valid = getFoldDataLoaders(timestamps,X,y,groups,trainIndex,validIndex)

            # supervised model instance and move to compute device
            model = wiFiFeaturesMLPModel(n_input=X.shape[1], n_output=3)
            model.to(device);
            # print(f"there are {find_no_of_trainable_params(model)} params in model")

            # optimizer function, lr schedulers and loss function
            optimizer = getOptimizer(model)
            scheduler = getScheduler(optimizer, dataloader_train)
            # print(f"optimizer={optimizer}, scheduler={scheduler}, loss_fn={criterion}")

            # train and validate single fold
            foldResults = trainValidateOneFold(buildingName, i_fold, model, optimizer,\
                                               scheduler,dataloader_train, dataloader_valid)
            buildingTrainResults = buildingTrainResults + foldResults
            
    buildingTrainResults = pd.DataFrame(buildingTrainResults)
    buildingTrainResults['valTotalLoss'] = buildingTrainResults['valPosLoss'] + buildingTrainResults['valFloorLoss']
    buildingTrainResults['trainTotalLoss'] = buildingTrainResults['trainPosLoss'] + buildingTrainResults['trainFloorLoss']
    
    return buildingTrainResults

In [ ]:
def getFoldBestResultsDf(trainResults):
    bestResults = []
    numFolds = trainResults['fold'].nunique()
    
    for fold in range(numFolds):
        foldDf = trainResults[trainResults['fold']== fold]
        bestResults.append(foldDf.iloc[np.argmin(foldDf['valTotalLoss'].values),:])
    
    bestResults =pd.DataFrame(bestResults)
    valPosLossBest = bestResults['valPosLoss'].values
    print(f"Best valPosLoss for all folds = {valPosLossBest}")
    print(f"Mean, std ={valPosLossBest.mean()}, {valPosLossBest.std()}")
    return bestResults

## Training & Validation main function

In [ ]:
%%time
if CFG.TRAIN == True:
    
    #for building in buildingsList:
    buildingTrainResults = trainValidateOneBuilding(buildingsList[0])
    bestResults = getFoldBestResultsDf(buildingTrainResults)
    # save results to csv
    buildingTrainResults.to_csv(f"{modelOutputDir}/{getBuildingName(buildingsList[0])}_{CFG.MODEL_NAME}_trainResults.csv")
    bestResults.to_csv(f"{modelOutputDir}/{getBuildingName(buildingsList[0])}_{CFG.MODEL_NAME}_bestResults.csv")
    
    # print and plot
    #print(buildingTrainResults.head(3))
    #print(bestResults.head(3))
    plotTrainingResults(buildingTrainResults)

## Generate OOF function

In [ ]:
def generateModelOOF():
    oof_ts = []
    oof_preds = []
    oof_targets = []
    oof_groups = []
    oof_folds = []

    buildingName = getBuildingName(buildingCsvPath=buildingsList[0])
    modelPaths = sorted(glob.glob(f"{modelOutputDir}/{buildingName}_{CFG.MODEL_NAME}_fold*.pth"))

    for fold in range(len(modelPaths)):
        # load building-model-fold checkpoint
        checkPoint = torch.load(modelPaths[fold])
        numRows = len(checkPoint['val_ts'])

        oof_ts.append(checkPoint['val_ts'])
        oof_preds.append(checkPoint['val_preds'])
        oof_targets.append(checkPoint['val_targets'])
        oof_groups.append(checkPoint['val_groups'])
        oof_folds.append([fold] * numRows)
    
    oof_ts = np.concatenate(oof_ts,axis=0)
    oof_preds = np.concatenate(oof_preds,axis=0)
    oof_targets = np.concatenate(oof_targets,axis=0)
    oof_groups = np.concatenate(oof_groups,axis=0)
    oof_folds = np.concatenate(oof_folds,axis=0)
    
    #print(oof_ts.shape, oof_preds.shape, oof_targets.shape, oof_groups.shape, oof_folds.shape)
    oof_df = pd.DataFrame({'ts' : oof_ts, 'x_preds': oof_preds[:,0], 'y_preds': oof_preds[:,1],
                       'floor_preds': oof_preds[:,2], 'x_tgt': oof_targets[:,0], 'y_tgt': oof_targets[:,1],
                       'floor_tgt': oof_targets[:,2], 'path' : oof_groups, 'fold' : oof_folds
                      })
    
    return oof_df